In [1]:
import win32com.client.gencache as gc
import win32com
c = win32com.client.constants

After a couple hours of trying everything I could think of, the following series of commands are the ONLY way I could figure out how to get data from the DataSocket to update. Specifically, The correct ReadMode, AccessMode must be used, and you must NOT call DataSocket.Update(). 

Other attempts resulted in complete failure to obtain data, obtain data only on 1st call, and in a couple places stalled the working thread. 

In [2]:
ds = gc.EnsureDispatch("CWDSLib.CWDataSocket")
ds.ReadMode = c.cwdsReadsSynchronous        
ds.SyncConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate, -1)
ds.SyncRead(-1)

False

In [3]:
oldvalue = -1

In [21]:
ds.SyncRead(-1)
#ds.Update()
#ds.SyncRead(-1)
value = ds.Data.Value
print(value == oldvalue)
oldvalue = value
value

False


(-0.9848077530122089,
 -0.9781476007336757,
 -0.9702957262761328,
 -0.9612616959383005,
 -0.9510565162955004,
 -0.9396926207860766,
 -0.9271838545667356,
 -0.9135454576430282,
 -0.8987940462993512,
 -0.8829475928588284,
 -0.8660254037849281,
 -0.8480480961566107,
 -0.8290375725548842,
 -0.8090169943754807,
 -0.7880107536068925,
 -0.766044443118751,
 -0.7431448254779534,
 -0.7193398003387937,
 -0.6946583704586917,
 -0.6691306063594259,
 -0.6427876096866417,
 -0.615661475325267,
 -0.5877852522930332,
 -0.5591929034707983,
 -0.5299192642327232,
 -0.5000000000005375,
 -0.4694715627858823,
 -0.4383711467885024,
 -0.40673664307630175,
 -0.37460659341583674,
 -0.3420201433267095,
 -0.3090169943754014,
 -0.2756373558168523,
 -0.2419218956006729,
 -0.2079116908181562,
 -0.17364817766670929,
 -0.13917310096102034,
 -0.10452846326798572,
 -0.0697564737438299,
 -0.03489949670339308,
 -2.6244047760987485e-13,
 0.034899496702868515,
 0.06975647374330629,
 0.10452846326746372,
 0.1391731009605006,
 0

In [8]:
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: TkAgg


In [24]:
f = plt.figure()
ax = f.add_subplot(111)
line, = ax.plot(ds.Data.Value)
f.show()

In [25]:
while True:
    ds.SyncRead(-1)
    y = ds.Data.Value
    x = list(range(len(y)))
    line.set_data(x, y)
    f.canvas.flush_events()
    f.canvas.draw()

TclError: this isn't a Tk application

In [ ]:
import win32com.client.gencache as gc
import win32com
c = win32com.client.constants

Just kidding, this way works too. 

http://zone.ni.com/reference/en-XX/help/375857A-01/html/t_nationalinstruments_net_readmode/
http://zone.ni.com/reference/en-XX/help/375857A-01/xml/usingdatasocketeventsaccessmodesnet/

tldr: in AutoUpdate access mode, the OnDataUpdated event is generated when data is changed. Otherwise, must use Update(). 
In ReadAsynchronous ReadMode, data is read automatically when updated from the server (aka pushed from server to client). Ohterwise, muse use SyncRead(). 

In [86]:
class EventHandler:
    def OnStatusUpdated(self, *args):
        #print("OnStatusUpdate")
        pass
    def OnDataUpdated(self, ob):
        ob.Invoke(1, 0, pythoncom.DISPATCH_PROPERTYGET, 1)
        print("test")

ds = gc.EnsureDispatch("CWDSLib.CWDataSocket")
handler = win32com.client.WithEvents(ds, EventHandler)
ds.ReadMode = c.cwdsReadsAsynchronous        
# ds.SyncConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate, -1)
ds.ConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate)
#ds.SyncRead(-1)
oldvalue = -1

test
test
test
test
test
test
test
test
test
test
test
test

In [87]:
#ds.SyncRead(-1)
#ds.Update()
#ds.SyncRead(-1)
value = ds.Data.Value
print(value == oldvalue)
oldvalue = value
value


False


(-5.848098380841784e-14,
 0.03489949670125447,
 0.06975647374532432,
 0.10452846326766656,
 0.13917310095890126,
 0.17364817766818458,
 0.20791169081784228,
 0.24192189559859653,
 0.2756373558182923,
 0.3090169943750962,
 0.34202014332469866,
 0.37460659341722574,
 0.4067366430760086,
 0.43837114678821393,
 0.4694715627839929,
 0.5000000000002596,
 0.529919264232451,
 0.5591929034690243,
 0.5877852522927736,
 0.6156614753250141,
 0.6427876096850025,
 0.6691306063591874,
 0.6946583704584608,
 0.7193398003373073,
 0.7431448254777386,
 0.7660444431185447,
 0.788010753605575,
 0.8090169943752921,
 0.8290375725547048,
 0.8480480961554767,
 0.8660254037847677,
 0.8829475928586777,
 0.8987940462984132,
 0.9135454576428976,
 0.9271838545666153,
 0.9396926207853448,
 0.9510565162954011,
 0.9612616959382121,
 0.9702957262756151,
 0.9781476007339871,
 0.9848077530121532,
 0.9902680687413609,
 0.9945218953683721,
 0.9975640502598072,
 0.9993908270190457,
 1.0,
 0.9993908270191018,
 0.9975640502599

test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


In [88]:
ds.Disconnect()

test


In [55]:
dir(lastargs[0])

['GetIDsOfNames',
 'GetTypeInfo',
 'GetTypeInfoCount',
 'Invoke',
 'InvokeTypes',
 'QueryInterface',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [63]:
lastarg = lastargs[0]
info = lastarg.GetTypeInfo()

In [64]:
dir(info)

['GetContainingTypeLib',
 'GetDocumentation',
 'GetFuncDesc',
 'GetIDsOfNames',
 'GetImplTypeFlags',
 'GetNames',
 'GetRefTypeInfo',
 'GetRefTypeOfImplType',
 'GetTypeAttr',
 'GetTypeComp',
 'GetVarDesc',
 'QueryInterface',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [73]:
info.GetNames(1)

('Value',)

In [74]:
lastarg.Value

AttributeError: 'PyIDispatch' object has no attribute 'Value'

In [79]:
import pythoncom

In [ ]:
pythoncom.DISPATCH_

In [80]:
lastarg.Invoke(1, 0, pythoncom.DISPATCH_PROPERTYGET, 1)

(0.3420201433265866,
 0.37460659341571545,
 0.4067366430745205,
 0.4383711467900197,
 0.46947156278576685,
 0.4999999999988489,
 0.5299192642341548,
 0.5591929034706898,
 0.5877852522914557,
 0.6156614753265973,
 0.6427876096865416,
 0.6691306063579769,
 0.6946583704599061,
 0.7193398003387029,
 0.7431448254766486,
 0.7660444431198361,
 0.7880107536068119,
 0.8090169943743346,
 0.8290375725558282,
 0.8480480961565414,
 0.8660254037839532,
 0.882947592857913,
 0.8987940462992939,
 0.9135454576422352,
 0.9271838545660052,
 0.9396926207860319,
 0.9510565162948978,
 0.961261695937763,
 0.9702957262761012,
 0.9781476007336485,
 0.9848077530118704,
 0.9902680687416405,
 0.9945218953682018,
 0.9975640502596936,
 0.9993908270191159,
 1.0,
 0.9993908270191586,
 0.9975640502597791,
 0.9945218953683299,
 0.9902680687418111,
 0.9848077530120831,
 0.9781476007339033,
 0.9702957262763976,
 0.9612616959381008,
 0.9510565162952765,
 0.939692620786451,
 0.9271838545664642,
 0.9135454576427335,
 0.89879